In [1]:
import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate


# Definition of the recommendation model
def create_recommendation_model(num_users, num_products, num_categories):
    # model inputs
    user_input = Input(shape=(num_users,), name='user_input')
    product_input = Input(shape=(num_products,), name='product_input')
    category_input = Input(shape=(num_categories,), name='category_input')

    # Embedding of users, products and categories
    embedding_dim = 50
    user_embedding = Embedding(num_users, embedding_dim, name='user_embedding')(user_input)
    product_embedding = Embedding(num_products, embedding_dim, name='product_embedding')(product_input)
    category_embedding = Embedding(num_categories, embedding_dim, name='category_embedding')(category_input)

    # Flatten the embedding layers
    user_flatten = Flatten()(user_embedding)
    product_flatten = Flatten()(product_embedding)
    category_flatten = Flatten()(category_embedding)

    # Concatenate the features
    merged_features = Concatenate()([user_flatten, product_flatten, category_flatten])

    # Layers of the neural network
    dense1 = Dense(128, activation='relu')(merged_features)
    dense2 = Dense(64, activation='relu')(dense1)
    output = Dense(1, activation='sigmoid')(dense2)

    # create the model
    model = Model(inputs=[user_input, product_input, category_input], outputs=output)
    
    return model




# Generation of Users
num_users = 100
users = [f"User_{i}" for i in range(num_users)]

#categories = ["Electronics", "Clothing", "Home", "Books"]
# one hot encoding function
def one_hot_encoding(T):
    types = np.unique(T) #books, clothing, electronics, home
    targets = np.eye(len(types))
    encoded_targets = np.zeros((len(T), len(types)))
    for i, item in enumerate(T):
        index = np.where(types == item)[0][0]
        encoded_targets[i] = targets[index]
    return encoded_targets

#print(one_hot_encoding(categories))

# Generation of products
num_products = 50
products = [
    {
        "id": f"Product_{i}",
        "name": f"Product {i}",
        "category": random.choice(["Electronics", "Clothing", "Home", "Books"]),
        "price": round(random.uniform(10, 1000), 2)
    }
    for i in range(num_products)
]

# Price normalization
scaler = MinMaxScaler()
precios = [product['price'] for product in products]
precios_array = np.array(precios).reshape(-1, 1)
precios_normalizados = scaler.fit_transform(precios_array)

# Update of normalized prices in the products
for i, product in enumerate(products):
    product['normalized_price'] = precios_normalizados[i][0]
    
# Combination of features
for product in products:
    product['combined_features'] = product['category'] + '_' + str(product['normalized_price'])

# Purchase history and website behavior
purchase_history = {}
site_behavior = {}

for user in users:
    # Generation of purchase history
    num_purchases = random.randint(1, 10)
    purchase_history[user] = random.sample(products, num_purchases)
    
    # Generation of website behavior
    num_visits = random.randint(10, 50)
    site_behavior[user] = {
        "visited_products": random.choices(products, k=num_visits)
    }

# Split the data into training and test sets
train_users, test_users = train_test_split(users, test_size=0.2, random_state=42)
train_products, test_products = train_test_split(products, test_size=0.2, random_state=42)
train_purchase_history = {user: purchase_history[user] for user in train_users}
test_purchase_history = {user: purchase_history[user] for user in test_users}
train_site_behavior = {user: site_behavior[user] for user in train_users}
test_site_behavior = {user: site_behavior[user] for user in test_users}

#print(len(train_users))
#print(len(test_users))

'''
# Print the generated data
print("Users:")
print(users)
print("\nProducts:")
for product in products:
    print(product)
print("\nPurchase History:")
for user, purchases in purchase_history.items():
    print(f"{user}: {purchases}")
print("\nSite Behavior:")
for user, behavior in site_behavior.items():
    print(f"{user}: {behavior}")

'''

# One-hot encoding for users
#encoded_users = one_hot_encoding(train_users)

#print(encoded_users)
# One-hot encoding for categories
categories = [product['category'] for product in train_products]
encoded_categories = one_hot_encoding(categories)
#print(categories)
#print(encoded_categories)

# Number of users, products and categories
num_users = len(train_users)
num_products = len(train_products)
num_categories = len(np.unique(categories))

# Create the recommendation model
recommendation_model = create_recommendation_model(num_users, num_products, num_categories)

# compile the model
recommendation_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print the model summary
recommendation_model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 user_input (InputLayer)        [(None, 80)]         0           []                               
                                                                                                  
 product_input (InputLayer)     [(None, 40)]         0           []                               
                                                                                                  
 category_input (InputLayer)    [(None, 4)]          0           []                               
                                                                                                  
 user_embedding (Embedding)     (None, 80, 50)       4000        ['user_input[0][0]']             
                                                                                              